In [1]:
import os
import random
import time
from typing import List, Tuple
from urllib.request import urlopen

import numpy as np
import scipy
import pandas as pd
import sklearn
import requests
import matplotlib.pyplot as plt

In [2]:
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor

In [13]:
import os
import random
import time
import json
from typing import Tuple
from urllib.request import urlopen

# ゲームサーバのアドレス
GAME_SERVER = os.getenv('GAME_SERVER', 'https://contest.2021-spring.gbc.tenka1.klab.jp')
# あなたのトークン
TOKEN = "b16c91e37d4c75777beba2eb755056fd"
# GameAPIを呼ぶ際のインターバル
GAME_INFO_SLEEP_TIME = 5000


def call_api(x: str) -> bytes:
    with urlopen(f'{GAME_SERVER}{x}') as res:
        return res.read()


def call_move(index: int, x: int, y: int) -> dict:
    res_str = call_api(f'/api/move/{TOKEN}/{index}-{x}-{y}')
    res_json = json.loads(res_str)
    return res_json


def call_move_next(index: int, x: int, y: int) -> dict:
    res_str = call_api(f'/api/move_next/{TOKEN}/{index}-{x}-{y}')
    res_json = json.loads(res_str)
    return res_json


def call_game() -> dict:
    res_str = call_api(f'/api/game/{TOKEN}')
    res_json = json.loads(res_str)
    return res_json


def call_master_data() -> dict:
    res_str = call_api(f'/api/master_data')
    res_json = json.loads(res_str)
    return res_json


class Bot:
    def __init__(self):
        self.move_histories = [["H", "O", "G", "E"] for _ in range(5)]

        self.master_data = call_master_data()
        self.game_info = call_game()
        self.start_game_time_ms = self.game_info['now']
        print("Start:", self.start_game_time_ms)
        self.start_time_ms = time.perf_counter_ns()//1000000
        self.next_call_game_info_time_ms = self.get_now_game_time_ms()+GAME_INFO_SLEEP_TIME
        self.agent_move_finish_ms = [0]*self.master_data['num_agent']
        self.agent_move_point_queue = [[] for _ in range(self.master_data['num_agent'])]
        self.agent_last_point = [[] for _ in range(self.master_data['num_agent'])]
        for i in range(self.master_data['num_agent']):
            agent_move = self.game_info['agent'][i]['move']
            self.agent_last_point[i] = [agent_move[-1]['x'], agent_move[-1]['y']]
            self.set_move_point(i)

    # タスクを取得
    def choice_task(self, index: int) -> str:
        tasks = self.game_info['task'][-18:]
        best_efficiency = 0
        best_task = ["Z"]
        
        
        totals = []
        for task in tasks:
            total = task["total"]
            totals.append(total)
        totals.sort()
        threshold = totals[len(totals)//3*2]
        
        
        for task1 in tasks:
            list_task1 = list(task1["s"])
            if random.random() < 0.999 and len(list_task1) == 5:
                continue
            if task1["total"] > threshold:
                continue
            if task1["s"] in {"LSBDZ", "EJHGQP", "KAHDBB"}:
                continue
            point1 = (1.0/(task1["total"]+1.0))**0.1
            tim1 = 0
            if self.move_histories[index][-4:] == list_task1[:4]:
                list_task1 = list_task1[4:]
            elif self.move_histories[index][-3:] == list_task1[:3]:
                list_task1 = list_task1[3:]
            elif self.move_histories[index][-2:] == list_task1[:2]:
                list_task1 = list_task1[2:]
            elif self.move_histories[index][-1:] == list_task1[:1]:
                list_task1 = list_task1[1:]
            else:
                pass
            c = self.move_histories[index][-1]
            x, y = self.get_checkpoint(c)
            for next_c in list_task1:
                next_x, next_y = self.get_checkpoint(next_c)
                t = max(0.5, 0.1*((x-next_x)**2+(y-next_y)**2)**0.5)
                tim1 += t

            # efficiency = point / tim
            # if best_efficiency < efficiency:
            #     best_efficiency = efficiency
            #     best_task = list_task1

            self.move_histories[index] += list_task1
            for task2 in tasks:
                list_task2 = list(task2["s"])
                point12 = point1+(1.0/(task2["total"]+1.0))**0.1
                tim12 = tim1
                if random.random() < 0.999 and len(list_task2) == 5:
                    continue
                if task2["total"] > threshold:
                    continue
                if task2["s"] == "LSBDZ":
                    continue
                if self.move_histories[index][-4:] == list_task2[:4]:
                    list_task2 = list_task2[4:]
                elif self.move_histories[index][-3:] == list_task2[:3]:
                    list_task2 = list_task2[3:]
                elif self.move_histories[index][-2:] == list_task2[:2]:
                    list_task2 = list_task2[2:]
                elif self.move_histories[index][-1:] == list_task2[:1]:
                    list_task2 = list_task2[1:]
                else:
                    pass
                c = self.move_histories[index][-1]
                x, y = self.get_checkpoint(c)
                for next_c in list_task2:
                    next_x, next_y = self.get_checkpoint(next_c)
                    t = max(1.0, 0.1*((x-next_x)**2+(y-next_y)**2)**0.5)
                    tim12 += t
                efficiency = point12/tim12
                if best_efficiency < efficiency:
                    best_efficiency = efficiency
                    best_task = list_task1
            for _ in list_task1:
                self.move_histories[index].pop()

        if random.random() < 0.03:
            best_task = list(random.choice(tasks)["s"])
        return "".join(best_task)

    def get_now_game_time_ms(self) -> int:
        now_ms = time.perf_counter_ns()//1000000
        return self.start_game_time_ms+(now_ms-self.start_time_ms)

    def get_checkpoint(self, name: str) -> Tuple[int, int]:
        index = ord(name)-ord('A')
        checkpoint = self.master_data['checkpoints'][index]
        return checkpoint['x'], checkpoint['y']

    # 移動予定を設定
    def set_move_point(self, index: int):
        next_task = self.choice_task(index)
        print("Agent#{} next task:{}".format(index, next_task))
        for i in range(len(next_task)):
            before_x = self.agent_last_point[index][0]
            before_y = self.agent_last_point[index][1]
            move_x, move_y = self.get_checkpoint(next_task[i])

            # 移動先が同じ場所の場合判定が入らないため別の箇所に移動してからにする
            if move_x == before_x and move_y == before_y:
                tmp_x = (before_x-1) if before_x > 10 else (before_x+1)
                tmp_y = before_y
                self.agent_move_point_queue[index].append([tmp_x, tmp_y])

            self.agent_move_point_queue[index].append([move_x, move_y])
            self.move_histories[index].append(next_task[i])
            self.move_histories[index] = self.move_histories[index][1:]
            self.agent_last_point[index] = [move_x, move_y]

    def move_next(self, index: int) -> dict:
        move_next_point = self.agent_move_point_queue[index].pop(0)
        move_next_res = call_move_next(index+1, move_next_point[0], move_next_point[1])
        print("Agent#{} move_next to ({}, {})".format(index+1, move_next_point[0], move_next_point[1]))

        if move_next_res['status'] != 'ok':
            print(move_next_res)
            exit(1)

        assert len(move_next_res['move']) > 1

        self.agent_move_finish_ms[index] = move_next_res['move'][1]['t']+100

        # タスクを全てやりきったら次のタスクを取得
        if not self.agent_move_point_queue[index]:
            self.set_move_point(index)

        return move_next_res

    # game_infoの状態でのスコアを取得
    def get_now_score(self) -> float:
        score = 0.0
        tasks = self.game_info['task']
        for i in range(len(tasks)):
            if tasks[i]['total'] == 0:
                continue
            score += tasks[i]['weight']*tasks[i]['count']/tasks[i]['total']

        return score

    def solve(self):
        while True:
            now_game_time_ms = self.get_now_game_time_ms()

            # エージェントを移動させる
            for i in range(self.master_data['num_agent']):
                if self.agent_move_finish_ms[i] < now_game_time_ms:
                    move_next_res = self.move_next(i)
                    # 次の移動予定がない場合もう一度実行する
                    if len(move_next_res['move']) == 2:
                        self.move_next(i)

            # ゲーム情報更新
            if self.next_call_game_info_time_ms < now_game_time_ms:
                print('Update GameInfo')
                self.game_info = call_game()
                self.next_call_game_info_time_ms = self.get_now_game_time_ms()+GAME_INFO_SLEEP_TIME
                print("Score: {}".format(self.get_now_score()))

            time.sleep(0.5)


In [14]:
bot = Bot()
bot.solve()

Start: 13565692
Agent#0 next task:XJPEXKNZ
Agent#1 next task:XJPEXKNZ
Agent#2 next task:XJPEXKNZ
Agent#3 next task:XJPEXKNZ
Agent#4 next task:XJPEXKNZ
Agent#1 move_next to (21, 0)
Agent#1 move_next to (29, 30)
Agent#2 move_next to (21, 0)
Agent#3 move_next to (21, 0)
Agent#4 move_next to (21, 0)
Agent#4 move_next to (29, 30)
Agent#5 move_next to (21, 0)
Agent#5 move_next to (29, 30)
Agent#2 move_next to (29, 30)
Agent#3 move_next to (29, 30)
Agent#4 move_next to (21, 23)
Agent#1 move_next to (21, 23)
Agent#5 move_next to (21, 23)
Agent#2 move_next to (21, 23)
Agent#3 move_next to (21, 23)
Agent#4 move_next to (25, 9)
Update GameInfo
Score: 15277.834578224418
Agent#4 move_next to (21, 0)
Agent#1 move_next to (25, 9)
Agent#5 move_next to (25, 9)
Agent#1 move_next to (21, 0)
Agent#2 move_next to (25, 9)
Agent#3 move_next to (25, 9)
Agent#4 move_next to (30, 6)
Agent#5 move_next to (21, 0)
Agent#2 move_next to (21, 0)
Agent#3 move_next to (21, 0)
Agent#4 move_next to (29, 13)
Agent#1 move_

Agent#2 move_next to (24, 17)
Agent#5 move_next to (8, 16)
Agent#3 move_next to (8, 16)
Agent#4 move_next to (3, 20)
Agent#2 move_next to (0, 14)
Agent#4 move_next to (15, 19)
Agent#1 move_next to (24, 17)
Agent#0 next task:XJPEXKNZ
Agent#1 move_next to (21, 0)
Agent#2 move_next to (8, 16)
Agent#5 move_next to (24, 17)
Agent#4 next task:XJPEXKNZ
Agent#3 move_next to (24, 17)
Agent#2 next task:XJPEXKNZ
Agent#5 move_next to (21, 0)
Agent#1 move_next to (29, 30)
Agent#3 move_next to (21, 0)
Agent#4 move_next to (14, 19)
Agent#5 move_next to (29, 30)
Update GameInfo
Score: 15283.736052144302
Agent#2 move_next to (24, 17)
Agent#1 next task:XJPEXKNZ
Agent#4 move_next to (15, 19)
Agent#4 move_next to (0, 14)
Agent#1 move_next to (21, 23)
Agent#2 move_next to (21, 0)
Agent#3 move_next to (29, 30)
Agent#5 move_next to (21, 23)
Agent#2 move_next to (29, 30)
Agent#3 move_next to (21, 23)
Agent#4 move_next to (12, 7)
Agent#3 next task:CUCKIYRI
Agent#4 move_next to (17, 4)
Agent#1 move_next to (25,

Agent#1 move_next to (24, 17)
Agent#4 move_next to (10, 20)
Agent#3 next task:VNEDUUYA
Agent#3 move_next to (30, 6)
Agent#4 move_next to (15, 24)
Agent#5 move_next to (24, 17)
Agent#1 move_next to (0, 14)
Agent#2 move_next to (3, 20)
Update GameInfo
Score: 15298.58387732073
Agent#2 move_next to (15, 19)
Agent#3 move_next to (24, 17)
Agent#5 move_next to (0, 14)
Agent#1 move_next to (8, 16)
Agent#4 move_next to (29, 13)
Agent#3 move_next to (0, 14)
Agent#4 move_next to (25, 9)
Agent#5 move_next to (8, 16)
Agent#2 move_next to (14, 19)
Agent#3 move_next to (8, 16)
Agent#1 move_next to (24, 17)
Agent#0 next task:XJPEXKNZ
Agent#1 move_next to (21, 0)
Agent#2 move_next to (15, 19)
Agent#2 move_next to (0, 14)
Agent#4 move_next to (3, 20)
Agent#4 move_next to (15, 19)
Agent#5 move_next to (24, 17)
Agent#4 next task:XJPEXKNZ
Agent#5 move_next to (21, 0)
Update GameInfo
Score: 15295.25299348137
Agent#3 move_next to (24, 17)
Agent#2 next task:XJPEXKNZ
Agent#1 move_next to (29, 30)
Agent#2 move_

Agent#2 move_next to (11, 11)
Agent#4 move_next to (25, 9)
Agent#5 move_next to (12, 7)
Agent#1 move_next to (29, 30)
Agent#3 move_next to (12, 7)
Agent#4 move_next to (21, 0)
Agent#2 move_next to (28, 18)
Agent#5 move_next to (29, 30)
Agent#4 move_next to (30, 6)
Update GameInfo
Score: 15332.518083593623
Agent#2 move_next to (10, 20)
Agent#3 move_next to (29, 30)
Agent#1 move_next to (28, 30)
Agent#4 move_next to (29, 13)
Agent#2 move_next to (15, 19)
Agent#4 move_next to (10, 20)
Agent#3 next task:WGZUAARL
Agent#5 move_next to (28, 30)
Agent#4 move_next to (11, 11)
Agent#3 move_next to (28, 30)
Agent#1 move_next to (29, 30)
Agent#1 move_next to (3, 20)
Agent#2 move_next to (12, 7)
Agent#2 move_next to (11, 7)
Agent#4 move_next to (28, 18)
Agent#5 move_next to (29, 30)
Agent#5 move_next to (3, 20)
Agent#4 move_next to (10, 20)
Update GameInfo
Score: 15325.362899797094
Agent#2 move_next to (12, 7)
Agent#2 move_next to (8, 16)
Agent#3 move_next to (29, 30)
Agent#3 move_next to (3, 20)
A

Agent#5 move_next to (29, 13)
Agent#1 move_next to (15, 19)
Agent#3 move_next to (29, 13)
Agent#4 move_next to (10, 20)
Agent#5 move_next to (0, 28)
Agent#1 move_next to (29, 13)
Agent#3 move_next to (0, 28)
Agent#1 move_next to (0, 28)
Agent#5 move_next to (0, 14)
Agent#4 next task:ZWUNSY
Agent#2 move_next to (30, 6)
Agent#4 move_next to (11, 11)
Agent#3 move_next to (0, 14)
Agent#2 next task:ZWUNSY
Agent#1 move_next to (0, 14)
Agent#0 next task:ZWUNSY
Agent#4 move_next to (15, 19)
Agent#2 move_next to (12, 7)
Update GameInfo
Score: 15987.692161972473
Agent#4 move_next to (29, 13)
Agent#4 move_next to (0, 28)
Agent#5 move_next to (10, 20)
Agent#3 move_next to (10, 20)
Agent#1 move_next to (10, 20)
Agent#2 move_next to (1, 10)
Agent#4 move_next to (0, 14)
Agent#3 next task:ZWUNSY
Agent#5 move_next to (11, 11)
Agent#3 move_next to (11, 11)
Agent#5 move_next to (15, 19)
Agent#1 move_next to (11, 11)
Agent#3 move_next to (15, 19)
Agent#2 move_next to (3, 20)
Agent#5 move_next to (29, 13)


...

Agent#3 move_next to (0, 14)
Agent#2 next task:ZWUNSY
Agent#1 move_next to (0, 14)
Agent#0 next task:ZWUNSY
Agent#5 move_next to (0, 14)
Agent#4 next task:ZWUNSY
Agent#2 move_next to (10, 20)
Agent#4 move_next to (11, 11)
Agent#4 move_next to (15, 19)
Agent#2 move_next to (11, 11)
Agent#4 move_next to (29, 13)
Update GameInfo
Score: 16312.401076565646
Agent#3 move_next to (10, 20)
Agent#2 move_next to (15, 19)
Agent#4 move_next to (0, 28)
Agent#5 move_next to (10, 20)
Agent#1 move_next to (10, 20)
Agent#2 move_next to (29, 13)
Agent#3 move_next to (11, 11)
Agent#1 move_next to (11, 11)
Agent#2 move_next to (0, 28)
Agent#4 move_next to (0, 14)
Agent#3 next task:ZWUNSY
Agent#5 move_next to (11, 11)
Agent#3 move_next to (15, 19)
Agent#1 move_next to (15, 19)
Agent#3 move_next to (29, 13)
Agent#5 move_next to (15, 19)
Agent#2 move_next to (0, 14)
Agent#1 next task:ZWUNSY
Agent#1 move_next to (29, 13)
Agent#3 move_next to (0, 28)
Agent#5 move_next to (29, 13)
Agent#5 move_next to (0, 28)
Up

Agent#3 move_next to (15, 19)
Agent#1 move_next to (15, 19)
Agent#5 move_next to (15, 19)
Agent#2 move_next to (0, 14)
Agent#1 next task:ZWUNSY
Agent#3 move_next to (29, 13)
Agent#1 move_next to (29, 13)
Agent#3 move_next to (0, 28)
Agent#5 move_next to (29, 13)
Agent#5 move_next to (0, 28)
Agent#1 move_next to (0, 28)
Agent#4 move_next to (10, 20)
Agent#3 move_next to (0, 14)
Agent#2 next task:ZWUNSY
Update GameInfo
Score: 16307.93883013062
Agent#1 move_next to (0, 14)
Agent#0 next task:ZWUNSY
Agent#5 move_next to (0, 14)
Agent#4 next task:ZWUNSY
Agent#2 move_next to (10, 20)
Agent#4 move_next to (11, 11)
Agent#2 move_next to (11, 11)
Agent#4 move_next to (15, 19)
Agent#4 move_next to (29, 13)
Agent#3 move_next to (10, 20)
Agent#2 move_next to (15, 19)
Agent#4 move_next to (0, 28)
Agent#5 move_next to (10, 20)
Agent#1 move_next to (10, 20)
Agent#2 move_next to (29, 13)
Agent#3 move_next to (11, 11)
Agent#1 move_next to (11, 11)
Agent#2 move_next to (0, 28)
Agent#4 move_next to (0, 14)

Agent#5 move_next to (10, 20)
Agent#3 move_next to (11, 11)
Agent#2 move_next to (1, 10)
Agent#4 move_next to (12, 7)
Agent#3 next task:ZWUNSY
Agent#1 move_next to (10, 20)
Agent#2 move_next to (30, 27)
Agent#5 move_next to (11, 11)
Agent#3 move_next to (15, 19)
Agent#1 move_next to (11, 11)
Agent#3 move_next to (29, 13)
Agent#5 move_next to (15, 19)
Update GameInfo
Score: 16260.612695917585
Agent#1 move_next to (15, 19)
Agent#3 move_next to (0, 28)
Agent#5 move_next to (29, 13)
Agent#1 move_next to (29, 13)
Agent#2 move_next to (15, 19)
Agent#4 move_next to (10, 20)
Agent#5 move_next to (0, 28)
Agent#1 move_next to (0, 28)
Agent#3 move_next to (0, 14)
Agent#2 next task:ZWUNSY
Agent#5 move_next to (0, 14)
Agent#4 next task:ZWUNSY
Agent#1 move_next to (0, 14)
Agent#0 next task:ZWUNSY
Agent#4 move_next to (11, 11)
Agent#2 move_next to (30, 6)
Agent#4 move_next to (15, 19)
Update GameInfo
Score: 16253.915531800707
Agent#3 move_next to (10, 20)
Agent#4 move_next to (29, 13)
Agent#5 move_ne

Agent#2 move_next to (10, 20)
Agent#3 move_next to (10, 20)
Agent#4 move_next to (11, 11)
Agent#1 move_next to (0, 14)
Agent#0 next task:ZWUNSY
Agent#4 move_next to (15, 19)
Agent#2 move_next to (11, 11)
Agent#3 move_next to (11, 11)
Update GameInfo
Score: 16260.343967835062
Agent#4 move_next to (29, 13)
Agent#5 move_next to (10, 20)
Agent#2 move_next to (15, 19)
Agent#3 move_next to (15, 19)
Agent#4 move_next to (0, 28)
Agent#2 move_next to (29, 13)
Agent#3 move_next to (29, 13)
Agent#5 move_next to (11, 11)
Agent#1 move_next to (10, 20)
Agent#2 move_next to (0, 28)
Agent#3 move_next to (0, 28)
Agent#4 move_next to (0, 14)
Agent#3 next task:ZWUNSY
Agent#5 move_next to (15, 19)
Agent#5 move_next to (29, 13)
Agent#1 move_next to (11, 11)
Agent#2 move_next to (0, 14)
Agent#1 next task:ZWUNSY
Agent#3 move_next to (0, 14)
Agent#2 next task:ZWUNSY
Agent#5 move_next to (0, 28)
Agent#1 move_next to (15, 19)
Update GameInfo
Score: 16248.751521046952
Agent#1 move_next to (29, 13)
Agent#4 move_n

KeyError: 'move'

In [6]:
random.random()

0.9575105958911774